===========================================


Gebil Jibul


Description: This program demonstrates the the deployment of a pre-trained deep learning image recognition model over the REST-based web service 'Falcon web server.'

=========================================== 

# Deploying Machine Learning Models

This project will create a REST-based web service that deploys a pre-trained deep learning image recognition model from Keras.  I will use the [Falcon Web Framework](https://falcon.readthedocs.io/en/stable/) to create a simple REST API and the [Requests library](https://docs.python-requests.org/en/latest/) to make API requests. 

I will use two Jupyter notebooks to implement the code for this assignment. `Project 12.ipynb` (this notebook) is where I implement the server-side code, and ` Project 12a.ipynb` is the client code used to test the server. The client-side code is for testing purposes only. 

Here I will create a REST-based web service that takes in an image URL as a parameter and returns a JSON-formatted response. 

In [17]:
from wsgiref.simple_server import make_server
import falcon

# Falcon follows the REST architectural style, meaning (among
# other things) that you think in terms of resources and state
# transitions, which map to HTTP verbs.
class ClassifyResourceV1:
    def on_get(self, req, resp):
        # Gets the img query parameter
        img_url = req.get_param('img')
        # Creates a result dictionary with a single entry
        result = dict(src_img=img_url)
        # Sets the response content to the result dictionary
        resp.media = result
        """Handles GET requests"""
        resp.status = falcon.HTTP_200  # This is the default status

        
# falcon.App instances are callable WSGI apps
# in larger applications the app is created in a separate file
app_v1 = falcon.App()

# Resources are represented by long-lived class instances
classify_v1 = ClassifyResourceV1()

# things will handle all requests to the '/classify' URL path
app_v1.add_route('/classify', classify_v1)

Below is a function that runs a Falcon app. I will use it to run `app_v1` and `app_v2`. 

In [18]:
def run_server(app):
    with make_server('', 9999, app) as httpd:
        print('Serving on port 9999...')

        # Serve until process is killed
        try:
            httpd.serve_forever()
        except KeyboardInterrupt:
            print('Stopping web server')

Run `run_server(app_v1)` in the cell below. This function will run until manually stopped. With this cell running, open `Assignment 12a_client.ipynb` and test the server. Once you finish testing, stop the cell from running. 

In [ ]:
run_server(app_v1)

<p>Serving on port 9999...</p>
<p>127.0.0.1 - - [24/Feb/2022 02:53:07] "GET /classify?img=https%3A%2F%2Fpicsum.photos%2F200%2F300 HTTP/1.1" 200 44</p>

Now I will implement a function that makes image classification predictions based on an image URL.  

In [19]:
from pathlib import Path
import tempfile

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

import numpy as np
import requests

import json

# Downloads and loads the ResNetModel
model = ResNet50(weights='imagenet')

The `load_img_url` function downloads an image to a temporary file and converts it into an object compatible with the ResNet50 model. 

In [25]:
def load_img_url(img_url, target_size):
    r = requests.get(img_url)
    if r.status_code != 200:
        print('Could not download image')
        return None
    
    img_suffix = '.{}'.format(img_url.split('.')[-1])
    
    with tempfile.NamedTemporaryFile(suffix=img_suffix, delete=False) as fp:
        fp.write(r.content)
        tf_path = Path(fp.name)
        img = image.load_img(tf_path, target_size=target_size)
    
    return img

In [26]:
def make_prediction(model, img_url, target_size=(224, 224), top=5):
    result = dict(src_img=img_url)
    img = load_img_url(img_url, target_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    preds = model.predict(x)
    
    result['predictions'] = []

    for class_name, class_description, score in decode_predictions(preds, top=top)[0]:
        result['predictions'].append(dict(
            class_name=str(class_name),
            class_description=str(class_description),
            score=float(score)
        ))
        
    return result    

Test the `make_prediction` function using a collection of image URLs.  Note: You may experience an error of the form `UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x15a8acc20>`.  Remove the offending image URL and try a different image. 

For testing purposes, this function outputs the following result for https://stardewvalleywiki.com/mediawiki/images/9/93/Greenhouse.png. 

```json
{
  "src_img": "https://stardewvalleywiki.com/mediawiki/images/9/93/Greenhouse.png",
  "predictions": [
    {
      "class_name": "n04201297",
      "class_description": "shoji",
      "score": 0.3795466423034668
    },
    {
      "class_name": "n02788148",
      "class_description": "bannister",
      "score": 0.1699788123369217
    },
    {
      "class_name": "n04589890",
      "class_description": "window_screen",
      "score": 0.07670347392559052
    },
    {
      "class_name": "n06359193",
      "class_description": "web_site",
      "score": 0.06704317778348923
    },
    {
      "class_name": "n04590129",
      "class_description": "window_shade",
      "score": 0.03521980345249176
    }
  ]
}
```

In [31]:
# Test the `make_prediction` function using img_urls
# Add img_urls for testing
img_urls = ['https://stardewvalleywiki.com/mediawiki/images/a/a4/Banner_Right_Winter.png',
            'https://www.stardewvalley.net/wp-content/uploads/2018/12/1_1screenshot15.png']

for img_url in img_urls:
    result = make_prediction(model, img_url)
    print('-----------------------------')
    print('')
    # Prints a pretty version of the result
    print(json.dumps(result, indent=2))
    print('-----------------------------')
    print()

-----------------------------

{
  "src_img": "https://stardewvalleywiki.com/mediawiki/images/a/a4/Banner_Right_Winter.png",
  "predictions": [
    {
      "class_name": "n03028079",
      "class_description": "church",
      "score": 0.6010048985481262
    },
    {
      "class_name": "n03854065",
      "class_description": "organ",
      "score": 0.26581016182899475
    },
    {
      "class_name": "n04366367",
      "class_description": "suspension_bridge",
      "score": 0.017505483701825142
    },
    {
      "class_name": "n04523525",
      "class_description": "vault",
      "score": 0.015335975214838982
    },
    {
      "class_name": "n02793495",
      "class_description": "barn",
      "score": 0.013667377643287182
    }
  ]
}
-----------------------------

-----------------------------

{
  "src_img": "https://www.stardewvalley.net/wp-content/uploads/2018/12/1_1screenshot15.png",
  "predictions": [
    {
      "class_name": "n03598930",
      "class_description": "jigsaw_pu

This part of the assignment combines the two previous steps. Implement `ClassifyResourceV2` that includes the`make_prediction` function. 

In [34]:
class ClassifyResourceV2:
    def on_get(self, req, resp):
        # TODO: Implement the code to handle the GET requests
        
        # Gets the img query parameter
        img_url = req.get_param('img')
        # Creates a result dictionary with a single entry
        result = dict(src_img=img_url)
        # Sets the response content to the result dictionary
        resp.media = result
        """Handles GET requests"""
        resp.status = falcon.HTTP_200  # This is the default status
        
        # Makes prediction and prints
        result = make_prediction(model, img_url)
        print('-'*25,'\n')
        # Prints a pretty version of the result
        print(json.dumps(result, indent=2))
        print('-'*25,'\n')


app_v2 = falcon.App()

classify_v2 = ClassifyResourceV2()

app_v2.add_route('/classify', classify_v2)

Once finished, run `app_v2` using the `run_server(app_v2)`.  This function will run until manually stopped. With this cell running, open 12a_client.ipynb and test the server. Once finish testing, stop the cell from running.

In [35]:
run_server(app_v2)

Serving on port 9999...
------------------------- 

{
  "src_img": "https://stardewvalleywiki.com/mediawiki/images/a/a4/Banner_Right_Winter.png",
  "predictions": [
    {
      "class_name": "n03028079",
      "class_description": "church",
      "score": 0.6010048985481262
    },
    {
      "class_name": "n03854065",
      "class_description": "organ",
      "score": 0.26581016182899475
    },
    {
      "class_name": "n04366367",
      "class_description": "suspension_bridge",
      "score": 0.017505483701825142
    },
    {
      "class_name": "n04523525",
      "class_description": "vault",
      "score": 0.015335975214838982
    },
    {
      "class_name": "n02793495",
      "class_description": "barn",
      "score": 0.013667377643287182
    }
  ]
}
------------------------- 



127.0.0.1 - - [24/Feb/2022 03:34:52] "GET /classify?img=https%3A%2F%2Fstardewvalleywiki.com%2Fmediawiki%2Fimages%2Fa%2Fa4%2FBanner_Right_Winter.png HTTP/1.1" 200 90


------------------------- 

{
  "src_img": "https://www.stardewvalley.net/wp-content/uploads/2018/12/1_1screenshot15.png",
  "predictions": [
    {
      "class_name": "n03598930",
      "class_description": "jigsaw_puzzle",
      "score": 0.985321581363678
    },
    {
      "class_name": "n02699494",
      "class_description": "altar",
      "score": 0.007495794910937548
    },
    {
      "class_name": "n03028079",
      "class_description": "church",
      "score": 0.0027745263651013374
    },
    {
      "class_name": "n03388043",
      "class_description": "fountain",
      "score": 0.002160582924261689
    },
    {
      "class_name": "n03877845",
      "class_description": "palace",
      "score": 0.0007866369560360909
    }
  ]
}
------------------------- 



127.0.0.1 - - [24/Feb/2022 03:35:04] "GET /classify?img=https%3A%2F%2Fwww.stardewvalley.net%2Fwp-content%2Fuploads%2F2018%2F12%2F1_1screenshot15.png HTTP/1.1" 200 91


Stopping web server


Serving on port 9999...
------------------------- 

{
  "src_img": "https://stardewvalleywiki.com/mediawiki/images/a/a4/Banner_Right_Winter.png",
  "predictions": [
    {
      "class_name": "n03028079",
      "class_description": "church",
      "score": 0.6010048985481262
    },
    {
      "class_name": "n03854065",
      "class_description": "organ",
      "score": 0.26581016182899475
    },
    {
      "class_name": "n04366367",
      "class_description": "suspension_bridge",
      "score": 0.017505483701825142
    },
    {
      "class_name": "n04523525",
      "class_description": "vault",
      "score": 0.015335975214838982
    },
    {
      "class_name": "n02793495",
      "class_description": "barn",
      "score": 0.013667377643287182
    }
  ]
}
------------------------- 

127.0.0.1 - - [24/Feb/2022 03:34:52] "GET /classify?img=https%3A%2F%2Fstardewvalleywiki.com%2Fmediawiki%2Fimages%2Fa%2Fa4%2FBanner_Right_Winter.png HTTP/1.1" 200 90
------------------------- 

{
  "src_img": "https://www.stardewvalley.net/wp-content/uploads/2018/12/1_1screenshot15.png",
  "predictions": [
    {
      "class_name": "n03598930",
      "class_description": "jigsaw_puzzle",
      "score": 0.985321581363678
    },
    {
      "class_name": "n02699494",
      "class_description": "altar",
      "score": 0.007495794910937548
    },
    {
      "class_name": "n03028079",
      "class_description": "church",
      "score": 0.0027745263651013374
    },
    {
      "class_name": "n03388043",
      "class_description": "fountain",
      "score": 0.002160582924261689
    },
    {
      "class_name": "n03877845",
      "class_description": "palace",
      "score": 0.0007866369560360909
    }
  ]
}
------------------------- 

127.0.0.1 - - [24/Feb/2022 03:35:04] "GET /classify?img=https%3A%2F%2Fwww.stardewvalley.net%2Fwp-content%2Fuploads%2F2018%2F12%2F1_1screenshot15.png HTTP/1.1" 200 91
Stopping web server